# Experiment 3 - LM with Sampled Data, Lowercased with SentencePiece Tokenization

In [5]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [6]:
from fastai.text import *
import html
import pickle
import sentencepiece as spm

In [8]:
PATH=Path('/home/paperspace/data/mimic-iii')
LM_PATH=PATH/'exp-3'

LM_PATH.mkdir(exist_ok=True)

In [11]:
df = pd.read_csv(PATH/'NOTEEVENTS.csv.gz')

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
notes = df.TEXT.str.lower().values

In [13]:
np.random.seed(42)
np.random.shuffle(notes)

In [14]:
trn_texts,val_texts = sklearn.model_selection.train_test_split(
    notes, test_size=0.1)

df_trn = pd.DataFrame({'text':trn_texts}, columns=['text'])
df_val = pd.DataFrame({'text':val_texts}, columns=['text'])

df_trn['text'] = df_trn.text.str.replace('\n','')
df_val['text'] = df_val.text.str.replace('\n','')

df_trn.to_csv(LM_PATH/'train.csv', header=False, index=False)
df_val.to_csv(LM_PATH/'test.csv', header=False, index=False)

In [21]:
#run this command from commandline, given how verbose it is
print(f'spm_train --input="{LM_PATH/"train.csv"}" --model_prefix="sp" --vocab_size=60000 --unk_id=0 --pad_id=1 --bos_id=2 --eos_id=3 --input_sentence_size=30000000 --num_threads=10')

spm_train --input="/home/paperspace/data/mimic-iii/exp-3/train.csv" --model_prefix="sp" --vocab_size=60000 --unk_id=0 --pad_id=1 --bos_id=2 --eos_id=3 --input_sentence_size=30000000 --num_threads=10


In [9]:
chunksize=20000

In [34]:
df_trn = pd.read_csv(LM_PATH/'train.csv', chunksize=chunksize, engine='python', header=None, names=['text'])
df_val = pd.read_csv(LM_PATH/'test.csv', chunksize=chunksize, engine='python', header=None, names=['text'])

In [117]:
sp = spm.SentencePieceProcessor()
sp.Load("sp.model")

def numericalize(df, partial = True):
    results = []
    
    for i, r in enumerate(df):
        print(i)
        
        results.append(r.text.apply(sp.EncodeAsIds).apply(lambda x: x + [3]).values) # 3 is the EOS ID
        
        if (i == 10) and (partial==True):
            break
            
    return np.concatenate(results)

In [ ]:
sp.I

In [107]:
trn_ids = numericalize(df_trn)
np.save(LM_PATH/'trn_ids.npy', trn_ids)

0
1
2
3
4
5
6
7
8
9
10


In [112]:
val_ids = numericalize(df_val)
np.save(LM_PATH/'val_ids.npy', val_ids)

0
1
2
3
4
5
6
7
8
9
10


In [115]:
trn_ids = np.load(LM_PATH/'trn_ids.npy')
np.save(LM_PATH/'trn_ids_concat.npy', np.concatenate(trn_ids))

In [116]:
val_ids = np.load(LM_PATH/'val_ids.npy')
np.save(LM_PATH/'val_ids_concat.npy', np.concatenate(val_ids))